In [2]:
import numpy as np
import numpy.linalg as lalg
from numpy.random import rand
import math
from s_gd2 import draw_svg, layout
import networkx as nx
from IPython.display import Image
from matplotlib.pyplot import imshow
%matplotlib inline

# generate graphs

In [3]:
"""
        0
      /   \
    1       2
   / \     / \
  3   4   5   6

children = x*2+1, x*2+2
parent = (x-1)/2
"""
def maketree(depth):
    I = []
    J = []
    def dfs(idx, depth):
        if depth <= 0:
            return
        parent = (idx-1)//2
        I.append(parent)
        J.append(idx)
        I.append(idx)
        J.append(parent)

        dfs(idx*2+1, depth-1)
        dfs(idx*2+2, depth-1)

    # node 0 is root and so has no parent
    dfs(1, depth-1)
    dfs(2, depth-1)
    return I,J

# make btree graph
depth = 7
I,J = maketree(depth)
n = (1<<depth)-1
A = np.zeros((n,n))
for i,j in zip(I,J):
    A[i,j] = 1

In [21]:
"""
   0 - 1 - 2 - 3
   |   |   |   |
   4 - 5 - 6 - 7
   |   |   |   |
   8 - 9 - 10- 11 
   |   |   |   |
   12- 13- 14- 15
"""

def two2oned(row, col):
    return row*width + col

def makegrid(width):
    I = []
    J = []
    for row in range(width):
        for col in range(width):
            if row>0:
                I.append(two2oned(row,col))
                J.append(two2oned(row-1,col))
            if row<width-1:
                I.append(two2oned(row,col))
                J.append(two2oned(row+1,col))
            if col>0:
                I.append(two2oned(row,col))
                J.append(two2oned(row,col-1))
            if col<width-1:
                I.append(two2oned(row,col))
                J.append(two2oned(row,col+1))
    return I,J

#make grid graph
width = 7
I,J = makegrid(width)
n = width*width
A = np.zeros((n,n))
for i,j in zip(I,J):
    A[i,j] = 1

In [65]:
# sierpinski
"""
        1
       / \
      2   3
     / \ / \
    4 - 5 - 6
   / \     / \
  7 - 8   9 - 0
 / \ / \ / \ / \
1 - 2 - 3 - 4 - 5

I have no idea what the actual indices will be lol
"""

def sierpinski(depth):
    idx = 0
    G = nx.Graph()
    def rec(depth):
        if depth <= 0:
            nonlocal idx
            nonlocal G
            G.add_edge(idx, idx+1)
            G.add_edge(idx+1, idx+2)
            G.add_edge(idx+2, idx)
            idx += 3
            return (idx-3, idx-2, idx-1)
        else:
            tri0 = rec(depth-1)
            tri1 = rec(depth-1)
            tri2 = rec(depth-1)

            G = nx.contracted_nodes(G, tri0[0], tri1[2])
            G = nx.contracted_nodes(G, tri1[1], tri2[0])
            G = nx.contracted_nodes(G, tri2[2], tri0[1])
            return (tri0[2], tri1[0], tri2[1])
        
    corners = rec(depth)
    G = nx.relabel.convert_node_labels_to_integers(G, label_attribute='orig')
    old2new = {b:a for a,b in nx.get_node_attributes(G, 'orig').items()}
    return G,(old2new[corners[0]], old2new[corners[1]], old2new[corners[2]])

G,corners = sierpinski(4)
A = nx.to_numpy_array(G)
I,J = zip(*G.edges)
n = len(G)

# do layouts

In [4]:
# calculate Laplacian matrix

D = np.identity(n) * np.sum(A, axis=1)
L = D - A

In [7]:
# Tutte tree: fix leaf vertices then solve

firstleaf = (1<<(depth-1))-1
print(f'{depth} {n} {firstleaf}')

b = np.zeros((n,2))
nleaves = (n-firstleaf)
for i in range(firstleaf, n):
    L[i,:] = 0
    L[i,i] = 1
    rad = (i-firstleaf)/nleaves * 2*math.pi
    b[i,0] = math.cos(rad)
    b[i,1] = math.sin(rad)

7 127 63


In [33]:
# Tutte grid: fix boundary vertices then solve
b = np.zeros((n,2))

def setlaplacianzero(idx, x, y):
    L[idx,:] = 0
    L[idx,idx] = 1
    b[idx,0] = x
    b[idx,1] = y
    
# spiral order hehe
# for col in range(width):
#     setlaplacianzero(two2oned(0,col), col, 0)
# for row in range(1,width):
#     setlaplacianzero(two2oned(row,width-1), width-1, row)
# for col in range(width-2,-1,-1):
#     setlaplacianzero(two2oned(width-1,col), col, width-1)
# for row in range(width-2,0,-1):
#     setlaplacianzero(two2oned(row,0), 0, row)

setlaplacianzero(two2oned(0,0), 0,0)
setlaplacianzero(two2oned(0,width-1), 0,1)
setlaplacianzero(two2oned(width-1,0), 1,0)
setlaplacianzero(two2oned(width-1,width-1), 1,1)

In [67]:
# Tutte Sierpinski: fix only corners
b = np.zeros((n,2))

i,j,k = corners
L[i,:] = L[j,:] = L[k,:] = 0
L[i,i] = L[j,j] = L[k,k] = 1

b[i] = [0,0]
b[j] = [1,0]
b[k] = [.5, .5*math.tan(math.pi/3)]

In [8]:
# solve tutte
X = lalg.solve(L,b)

draw_svg(X, I, J, 'figures/force_tutte.svg', linkwidth=.003, noderadius=.006)

In [59]:
# Spectral: calculate eigenvalues
w,v = lalg.eig(L)

# take second and third eigenvectors
idx = w.argsort()
X = np.real(v[:,idx[1:3]]) # 0 index is zero eigenvalue

draw_svg(X, I, J, 'figures/force_spectral.svg', linkwidth=.002, noderadius=.003)

In [80]:
# Eades
c1 = 2
c2 = 1
c3 = 1
c4 = .1
iters = 100

X = rand(n,2) * 6 # if not multiplied by six, the repulsive force are too big
for i in range(iters):
    # sum up gradients
    ddx = np.zeros((n,2))
    for src in range(n):
        for tgt in range(n):
            if src==tgt:
                continue
            vec = X[tgt] - X[src]
            d = lalg.norm(vec)
            norm = vec/d
            if A[src,tgt] != 0:
                ddx[src] -= (c1 * math.log2(d/c2)) * norm
            else:
                ddx[src] += (c3 / (d*d)) * norm
    # descend
    for src in range(n):
        X[src] -= c4 * ddx[src]
        
draw_svg(X, I, J, 'figures/force_eades.svg', linkwidth=.2, noderadius=.4)

In [78]:
# stress
X = layout(I,J)
draw_svg(X, I, J, 'figures/force_stress.svg', linkwidth=.1, noderadius=.2)